In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
import sys

In [2]:
class Node():
    def __init__(self, name, support):
        self.name = name
        self.childs = []
        self.support = support
        
    def add(self, child):
        self.childs.append(child)
        
        
class FP_tree():
    
    def __init__(self, data, items, support):
        self.data = data
        self.transformed_data = data
        self.support = support
        self.freq_dict = dict()
        self.items = items
        self.root = Node(None, -1)
        self.conditional_trees = dict()
        self.itemsets = dict()
        self.freq_itemsets = []
        
        self.__get_freq_of_individual_items()
        
    def __get_freq_of_individual_items(self):
        
        for i, item in enumerate(self.items):
            temp = self.data[:, i]
            freq = len(np.where(temp == 1)[0])
            if freq >= self.support:
                self.freq_dict[item] = freq
                continue
                
            self.transformed_data[:,i] = 0
            
        self.freq_dict = {k: v for k, v in sorted(self.freq_dict.items(), key=lambda item: item[1])}   
        self.transformed_data = self.transformed_data[~np.all(self.transformed_data == 0, axis=1)]
            
    def __add_nodes(self, node, t, i=0):
        
        if i == len(t):
            return
        name = self.items[t[i]]
        child_node = Node(name, 1)
#         print("parent_node: ", node.name, "child_node: ", child_node.name)
        node.add(child_node)
        node.support += 1
        self.__add_nodes(child_node, t, i+1)
        
        
    def construct_tree(self):
        
        for data in tqdm(self.transformed_data):
            
            t = np.where(data == 1)[0]
            node = self.__check_for_existing_branch(self.root, t)
            if node.name is None:
                self.__add_nodes(node, t)
            
            elif node.name != self.items[t[-1]]:
                item_index = np.where(self.items == node.name)[0][0]
                
                i = np.where(t == item_index)[0][0]
                
                self.__add_nodes(node, t, i + 1)
            
        return self.root
            
    
    def __check_for_existing_branch(self, node, t, i=0):
        if not node.childs or i == len(t):
            return node
        
        childs = [child.name for child in node.childs]
        
        item = self.items[t[i]]
        
        if item not in childs:
            return node
        
        next_node = next((child for child in node.childs if child.name == item), None)
        next_node.support += 1
        return self.__check_for_existing_branch(next_node, t, i+1)
    
    def __find_node_path(self, item, node, path = []):
        if node.name == item:
            return node
        
        if not node.childs:
            return None
        
        next_node = next((child for child in node.childs if child.name == item), None)
        
        path.append(node)
        if not next_node:
            
            for child in node.childs:
                return self.__find_node_path(item, child, path)
        else:
                
            path.append(next_node)
            
            return path
        
    
    def construct_conditional_FP_trees(self):
        
        for item in self.freq_dict:
            paths = []
            for child in self.root.childs: 
                path = self.__find_node_path(item, child, [])
                if path:
                    if isinstance(path, list):
                        support = path[-1].support
                    else:
                        support = path.support
                    paths.append((support, path))
                    
            self.conditional_trees[item] = paths
            
            items = []
            for support, path in paths:
                if isinstance(path, list):
                    l = [p.name for p in path]
                    items.append((support, l))
                    continue
                items.append((support, path.name))
                
                
            self.itemsets[item] = items
                
        return self.conditional_trees, self.get_frequent_itemsets()
    
    def get_frequent_itemsets(self):
        
        for item in self.itemsets:
            itemset = self.itemsets[item]
            
            for pitem in itemset:
                support = pitem[0]
                if support >= self.support:
                    self.freq_itemsets.append(pitem)
                    
        return self.freq_itemsets
        

#### Utils

In [13]:
def load(path, dtype=int):
    file = open(path, 'r')
    data = [[dtype(x) for x in line.split()] for line in file]
    
    file.close()
    
    return data

def one_hot_encode(data, length):
    no_of_transaction = len(data)
    
    one_hot_encode = np.zeros((no_of_transaction, length), dtype=np.uint8)
    
    for i,row in enumerate(data):
        for index in row:
            one_hot_encode[i, index] = 1
    return one_hot_encode

def write(freq_itemsets, topic):
    dir = 'results'
    save_path = os.path.join(dir, "PATTERN-{}.txt".format(topic))
    with open(save_path, 'w') as f:
        for item in freq_itemsets:
            support = item[0]
            result = str(support)
            if isinstance(item[1], list):
                for i in item:
                    result = result + ' ' + i
                    
            else:
                result = result + ' ' + item[1]
            f.write("%s\n" % result)

In [3]:
vocabs = np.array(load('data/vocab.txt', str))
length = vocabs.shape[0]
topic_0 = one_hot_encode(load('data/topic-0.txt'), length)
topic_1 = one_hot_encode(load('data/topic-1.txt'), length)
topic_2 = one_hot_encode(load('data/topic-2.txt'), length)
topic_3 = one_hot_encode(load('data/topic-3.txt'), length)
topic_4 = one_hot_encode(load('data/topic-4.txt'), length)

In [4]:
fp_tree0 = FP_tree(topic_0, vocabs[:,1], 400)
fp_tree1 = FP_tree(topic_1, vocabs[:,1], 400)
fp_tree2 = FP_tree(topic_2, vocabs[:,1], 400)
fp_tree3 = FP_tree(topic_3, vocabs[:,1], 400)
fp_tree4 = FP_tree(topic_4, vocabs[:,1], 400)

In [5]:
tree0 = fp_tree0.construct_tree()
tree1 = fp_tree1.construct_tree()
tree2 = fp_tree2.construct_tree()
tree3 = fp_tree3.construct_tree()
tree4 = fp_tree4.construct_tree()

In [6]:
cd_tree0, freq_itemsets0 = fp_tree0.construct_conditional_FP_trees()
cd_tree1, freq_itemsets1 = fp_tree1.construct_conditional_FP_trees()
cd_tree2, freq_itemsets2 = fp_tree2.construct_conditional_FP_trees()
cd_tree3, freq_itemsets3 = fp_tree3.construct_conditional_FP_trees()
cd_tree4, freq_itemsets4 = fp_tree4.construct_conditional_FP_trees()

In [17]:
write(freq_itemsets0, 0)
write(freq_itemsets1, 1)
write(freq_itemsets2, 2)
write(freq_itemsets3, 3)
write(freq_itemsets4, 4)